In [28]:
import requests
from bs4 import BeautifulSoup
import lxml
import pandas as pd
import time
import json
import pathlib

In [8]:
session = requests.Session()

In [9]:
session.headers

{'User-Agent': 'python-requests/2.31.0', 'Accept-Encoding': 'gzip, deflate, br', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [10]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; en-US; rv:76.0) Gecko/20100101 Firefox/76.0'}
session.headers.update(headers)

In [11]:
params_tmccmldm = {
    'fillQuickSearch':'false',
    'target':'advanced',
    'expand':'dl',
    'field1':'AllField',
    'text1':"Text AND Mining OR Clustering OR Classification OR Machine AND Learning OR Data AND Mining",
    'Ppub':"[20170315 TO 20220315]",
    'pageSize':50,
    'startPage':0,
    'rel':'nofollow',
    'ContentItemType':'research-article'
}
params_ha = {'fillQuickSearch':'false',
              'target':'advanced',
              'expand':'dl',
              'field1':'AllField',
              'text1':"Hardware AND Architecture",
              'Ppub':"[20210315 TO 20220315]",
              'pageSize':50,
              'startPage':0,
              'rel':'nofollow',
              'ContentItemType':'research-article'}
params_se = {'fillQuickSearch':'false',
              'target':'advanced',
              'expand':'dl',
              'field1':'AllField',
              'text1':"Software AND Engineering",
              'Ppub':"[20210315 TO 20220315]",
              'pageSize':50,
              'startPage':0,
              'rel':'nofollow',
              'ContentItemType':'research-article'}
params_fl = {'fillQuickSearch':'false',
              'target':'advanced',
              'expand':'dl',
              'field1':'AllField',
              'text1':"Fuzzy AND Logic",
              'Ppub':"[20210315 TO 20220315]",
              'pageSize':50,
              'startPage':0,
              'rel':'nofollow',
              'ContentItemType':'research-article'}
params_sc = {'fillQuickSearch':'false',
              'target':'advanced',
              'expand':'dl',
              'field1':'AllField',
              'text1':"Systems AND Control",
              'Ppub':"[20210315 TO 20220315]",
              'pageSize':50,
              'startPage':0,
              'rel':'nofollow',
              'ContentItemType':'research-article'}
params_db = {'fillQuickSearch':'false',
              'target':'advanced',
              'expand':'dl',
              'field1':'AllField',
              'text1':"Databases",
              'Ppub':"[20210315 TO 20220315]",
              'pageSize':50,
              'startPage':0,
              'rel':'nofollow',
              'ContentItemType':'research-article'}
params_cs = {'fillQuickSearch':'false',
              'target':'advanced',
              'expand':'dl',
              'field1':'AllField',
              'text1':"Computer AND Security",
              'Ppub':"[20210315 TO 20220315]",
              'pageSize':50,
              'startPage':0,
              'rel':'nofollow',
              'ContentItemType':'research-article'}

In [12]:
response = session.get('https://dl.acm.org/action/doSearch', params=params_tmccmldm)

In [13]:
response.status_code

200

In [14]:
response.url

'https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&expand=dl&field1=AllField&text1=Text+AND+Mining+OR+Clustering+OR+Classification+OR+Machine+AND+Learning+OR+Data+AND+Mining&Ppub=%5B20170315+TO+20220315%5D&pageSize=50&startPage=0&rel=nofollow&ContentItemType=research-article'

In [15]:
def parse_page(params, page):
    params['startPage'] = page
    response = session.get('https://dl.acm.org/action/doSearch', params=params)
    soup = BeautifulSoup(response.text, 'lxml')
    titles_links = [[title.text, f"https://dl.acm.org{title.select_one('a')['href']}"] for title in soup.select('.hlFld-Title')]
    return titles_links

In [37]:
def parse_info(url):
    response = session.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    authors = [author.text for author in soup.select('.loa__author-name')]
    meta = soup.select_one('.issue-item__detail a')
    meta = meta['title'] if meta else None
    date = soup.select_one('.epub-section__date')
    date = date.text.strip() if date else None
    abstract = soup.select_one('.abstractSection')
    abstract = abstract.text.strip() if abstract else None
    return authors, meta, date, abstract

In [ ]:
params = [params_tmccmldm, params_ha, params_se, params_fl, params_sc, params_db, params_cs]
categories = ['tmccmldm', 'ha', 'se', 'fl', 'sc', 'db', 'cs']
for i in range(1, len(params)):
    with open(f'{categories[i]}_category.csv', 'a', newline='', encoding='utf-8') as file:
        file.write(f"title;;;authors;;;date;;;source;;;abstract;;;link;;;category\n")
        for page in range(20):
            titles_links = parse_page(params[i], page)
            for title, link in titles_links:
                authors, source, date, abstract = parse_info(link)
                file.write(f"{title};;;{authors};;;{date};;;{source};;;{abstract};;;{link};;;{params[i]['text1']}\n")

In [58]:
df = pd.read_csv('tmccmldm_category.csv', encoding='utf-8', sep=';;;')
df

C:\Users\linag\AppData\Local\Temp\ipykernel_6968\3011047585.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('tmccmldm_category.csv', encoding='utf-8', sep=';;;')


,title,authors,date,source,abstract,link,category
0,Text mining for incoming tasks based on the ur...,['Yasser Ali Alshehri'],March 2020,ICCDA '20: Proceedings of the 2020 4th Interna...,"In workplaces, there is a massive amount of un...",https://dl.acm.org/doi/10.1145/3388142.3388153,Text AND Mining OR Clustering OR Classificatio...
1,Text mining for malware classification using m...,"['Konstantinos F. Xylogiannopoulos', 'Panagiot...",August 2019,ASONAM '19: Proceedings of the 2019 IEEE/ACM I...,Mobile phones have become nowadays a commodity...,https://dl.acm.org/doi/10.1145/3341161.3350841,Text AND Mining OR Clustering OR Classificatio...
2,Comparative Study between Traditional Machine ...,"['Cannannore Nidhi Kamath', 'Syed Saqib Bukhar...",August 2018,DocEng '18: Proceedings of the ACM Symposium o...,"In this contemporaneous world, it is an obliga...",https://dl.acm.org/doi/10.1145/3209280.3209526,Text AND Mining OR Clustering OR Classificatio...
3,Where is the road for issue reports classifica...,"['Qiang Fan', 'Yue Yu', 'Gang Yin', 'Tao Wang'...",November 2017,ESEM '17: Proceedings of the 11th ACM/IEEE Int...,"Currently, open source projects receive variou...",https://dl.acm.org/doi/10.1109/ESEM.2017.19,Text AND Mining OR Clustering OR Classificatio...
4,Identification of Overpricing in the Purchase ...,"['Marco Aurelio O. S. Correa', 'Adriano Galind...",August 2018,ICCBDC '18: Proceedings of the 2018 2nd Intern...,Increasing the transparency level in his actio...,https://dl.acm.org/doi/10.1145/3264560.3264569,Text AND Mining OR Clustering OR Classificatio...
...,...,...,...,...,...,...,...
996,TipTap: Approximate Mining of Frequent k-Subgr...,"['Muhammad Anis Uddin Nasir', 'Cigdem Aslay', ...",NaN,ACM Transactions on Knowledge Discovery from Data,“Perhaps he could dance first and think afterw...,https://dl.acm.org/doi/10.1145/3442590,Text AND Mining OR Clustering OR Classificatio...
997,Interactive Document Clustering Revisited: A V...,"['Ehsan Sherkat', 'Seyednaser Nourashrafeddin'...",March 2018,IUI '18: 23rd International Conference on Inte...,Document clustering is an efficient way to get...,https://dl.acm.org/doi/10.1145/3172944.3172964,Text AND Mining OR Clustering OR Classificatio...
998,Data Mining in Health Care Sector: Literature ...,"['Ahed Abugabah', 'Ahmad Al Smadi', 'Alaa Abuq...",November 2019,CIIS '19: Proceedings of the 2019 2nd Internat...,A standout amongst the most essential strides ...,https://dl.acm.org/doi/10.1145/3372422.3372451,Text AND Mining OR Clustering OR Classificatio...
999,Classification of Bacterial and Viral Childhoo...,"['Xianghong Gu', 'Liyan Pan', 'Huiying Liang',...",March 2018,ICMIP '18: Proceedings of the 3rd Internationa...,"Over decades, computer aided diagnosis (CAD) s...",https://dl.acm.org/doi/10.1145/3195588.3195597,Text AND Mining OR Clustering OR Classificatio...


In [29]:
def csv2json(csv_file_path, json_file_path, delimiter):
    json_data = []

    with open(csv_file_path, 'r', encoding='utf-8') as file:
        headers = file.readline().strip().split(delimiter)

        for line in file:
            values = line.strip().split(delimiter)
            if len(values) != len(headers):
                continue
            data = {headers[i]: values[i] for i in range(len(headers))}
            json_data.append(data)

    with open(json_file_path, 'w', encoding='utf-8') as json_file:
        json.dump(json_data, json_file, indent=4, ensure_ascii=False)

In [27]:
csv2json('tmccmldm_category.csv', 'tmccmldm_category.json', ';;;')

In [31]:
csv2json('ha_category.csv', 'ha_category.json', ';;;')

In [32]:
csv2json('se_category.csv', 'se_category.json', ';;;')

In [33]:
csv2json('fl_category.csv', 'fl_category.json', ';;;')

In [34]:
csv2json('sc_category.csv', 'sc_category.json', ';;;')

In [35]:
csv2json('db_category.csv', 'db_category.json', ';;;')

In [36]:
csv2json('cs_category.csv', 'cs_category.json', ';;;')